In [2]:
import os
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
from scipy.stats import pearsonr
from sklearn.metrics import mean_squared_error

sys.path.insert(0, "../../src")
from scripts.DSI.DS import DempsterShafer
from scripts.DSI.DSI_input import DempsterShaferInputHandler


# Load data

In [11]:
# import pandas as pd
# import numpy as np

# # Define the deep learning models
# models = ['1D CNN', 'LSTM', 'BiLSTM', 'CNN-LSTM', 'GRU', 'ANN']

# # Generate random values for the 'r' column
# r_values = np.random.uniform(-1, 1, len(models))

# # Create the dataframe
# df = pd.DataFrame({'rmse': np.random.uniform(0, 5, len(models)), 'r': r_values}, index=models)
# df.index.name = 'Model'

# # Print the dataframe
# print(df)
# df.to_csv("../../data/nowcast_dsi.csv", index=True, header=True)


In [3]:
nowcast_multimodel_data = pd.read_csv("../../data/nowcast_dsi.csv", index_col="Model")
parameters = ["RMSE", "Pearson_r"]
nowcast_multimodel_data.columns = parameters
nowcast_multimodel_data


,RMSE,Pearson_r
Model,,
1D CNN,4.041753,0.015509
LSTM,4.372815,-0.276929
BiLSTM,4.450400,-0.641712
CNN-LSTM,1.982408,0.858672
GRU,4.186350,0.112732
ANN,3.640281,0.231497


# Ensemble

In [20]:
def calculate_statistics(y_true, y_pred):
    stats = {}
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    # mae = mean_absolute_error(y_true, y_pred)
    pearson_r, _ = pearsonr(y_true, y_pred)
    # r2 = r2_score(y_true, y_pred)
    # max_error_value = max_error(y_true, y_pred)
    # mape = mean_absolute_percentage_error(y_true, y_pred)
    # return rmse, mae, pearson_r, r2, max_error_value, mape
    stats["RMSE"] = rmse
    stats["Pearson_r"] = pearson_r
    return stats


def evaluate_forecast_models(forecast_dict, observation_df):
    """
    Evaluate multiple forecast models against actual observations.

    Args:
    - forecast_dict: Dictionary of forecasted values with the format:
        {"forecast_hour": {"model1": forecast_df1, "model2": forecast_df2, ...}}
    - observation_df: DataFrame with the actual observed values, indexed by time.

    Returns:
    - A dictionary with the calculated statistics for each model.
    """
    results = {}
    forecast_length = len(observation_df)
    for forecast_hour, models in forecast_dict.items():
        model_stats = {}
        model_stats_df = pd.DataFrame()
        for model_name, forecast_df in models.items():
            if len(forecast_df) < len(observation_df):
                raise ValueError("Forecast and observation lengths do not match.")
            # Align forecast and observation data based on DateTimeIndex
            common_index = forecast_df.index.intersection(observation_df.index)

            # Ensure forecast values cover the desired observation period
            relevant_forecast = forecast_df.loc[common_index[:forecast_length]]
            relevant_observation = observation_df.loc[common_index[:forecast_length]]

            # Calculate statistics only if there are enough data points
            if len(relevant_observation) == forecast_length:
                stats = calculate_statistics(
                    relevant_forecast.values.flatten(),
                    relevant_observation.values.flatten(),
                )
                model_stats[model_name] = stats
                if model_stats.empty:
                    model_stats = pd.DataFrame.from_dict(model_stats, orient="index")
                else:
                    model_stats = pd.concat([model_stats_df, model_stats], axis=1)

        results[forecast_hour] = model_stats

    return results


# Example usage
# forecasts = {"6": {"LSTM": lstm_forecast, "CNN": cnn_forecast}, "12": {"LSTM": lstm_forecast_12, "CNN": cnn_forecast_12}}
# observations = actual_observations
# stats = calculate_statistics(forecasts, observations)


## Dummy test

In [21]:
# # Function to generate dummy forecast and observation data
# def generate_dummy_data(start_time, periods, freq, num_models, forecast_length):
#     # Create a DateTimeIndex
#     time_index = pd.date_range(start=start_time, periods=periods, freq=freq)

#     # Generate random data for forecasts
#     forecast_dict = {}
#     for forecast_hour in [6, 12, 24]:  # Assuming different forecast lengths
#         models = {}
#         for i in range(1, num_models + 1):
#             model_name = f"Model_{i}"
#             # Generate forecast values with slight variations per model
#             forecast_values = np.random.rand(periods) * 100 + i * 5
#             models[model_name] = pd.DataFrame(
#                 forecast_values, index=time_index, columns=["forecast"]
#             )
#         forecast_dict[str(forecast_hour)] = models

#     # Generate observation data
#     observation_values = np.random.rand(periods) * 100
#     observation_df = pd.DataFrame(
#         observation_values, index=time_index, columns=["observed"]
#     )

#     return forecast_dict, observation_df


# # Parameters
# start_time = "2024-08-01 00:00:00"
# periods = 48  # Total number of hours
# freq = "H"  # Hourly data
# num_models = 3  # Number of models
# forecast_length = 6  # Number of hours to evaluate

# # Generate dummy data
# forecast_dict, observation_df = generate_dummy_data(
#     start_time, periods, freq, num_models, forecast_length
# )

# # Test the evaluate_forecast_models function
# evaluation_results = evaluate_forecast_models(forecast_dict, observation_df)

# # Print the evaluation results
# print("Evaluation Results:", evaluation_results)
# for forecast_hour, model_stats in evaluation_results.items():
#     print(f"\nForecast Hour: {forecast_hour}")
#     for model_name, stats in model_stats.items():
#         print(
#             f"  {model_name} -> RMSE: {stats['RMSE']:.3f}, Pearson_r: {stats['Pearson_r']:.3f}"
#         )


Evaluation Results: {'6': {'Model_1': {'RMSE': 46.06055056865524, 'Pearson_r': -0.1500307744453881}, 'Model_2': {'RMSE': 45.61961229922331, 'Pearson_r': -0.030381329066097928}, 'Model_3': {'RMSE': 47.59428055705401, 'Pearson_r': -0.2779249015068874}}, '12': {'Model_1': {'RMSE': 38.219700977593845, 'Pearson_r': 0.12163626774227945}, 'Model_2': {'RMSE': 46.14193596110504, 'Pearson_r': -0.06883608660582284}, 'Model_3': {'RMSE': 48.41753465101365, 'Pearson_r': -0.19993383748272897}}, '24': {'Model_1': {'RMSE': 38.407759765219204, 'Pearson_r': 0.18136988825913675}, 'Model_2': {'RMSE': 44.390708709699915, 'Pearson_r': -0.08173553753997634}, 'Model_3': {'RMSE': 43.91802354570127, 'Pearson_r': 0.21394501245327968}}}

Forecast Hour: 6
  Model_1 -> RMSE: 46.061, Pearson_r: -0.150
  Model_2 -> RMSE: 45.620, Pearson_r: -0.030
  Model_3 -> RMSE: 47.594, Pearson_r: -0.278

Forecast Hour: 12
  Model_1 -> RMSE: 38.220, Pearson_r: 0.122
  Model_2 -> RMSE: 46.142, Pearson_r: -0.069
  Model_3 -> RMSE: 48

C:\Users\13993807_admin\AppData\Local\Temp\ipykernel_6252\573354599.py:4: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  time_index = pd.date_range(start=start_time, periods=periods, freq=freq)


# DSI

## Create matrices

In [ ]:
sampling_matrix = np.array([[0, 1]])
print(sampling_matrix.shape)
reference_matrix = nowcast_multimodel_data[parameters].values
reference_matrix.shape
reference_matrix = pd.DataFrame(reference_matrix, columns=parameters)
sampling_matrix = pd.DataFrame(sampling_matrix, columns=parameters)


(1, 2)


In [ ]:
test_hypotheses = list(nowcast_multimodel_data.index)
test_hypotheses


['1D CNN', 'LSTM', 'BiLSTM', 'CNN-LSTM', 'GRU', 'ANN']

In [ ]:
# Define the Dempster-Shafer input handler
ds_handler = DempsterShaferInputHandler(reference_matrix, sampling_matrix)
ds_handler.normalize_data()
normalized_reference = ds_handler.get_normalized_reference_matrix()
normalized_sampling = ds_handler.get_normalized_sampling_matrix()

normalized_reference_np = normalized_reference.to_numpy()
normalized_sampling_np = normalized_sampling.to_numpy()
print(normalized_reference_np)
print(normalized_sampling_np)

is_uncertain = "Uncertain" in test_hypotheses
dsi = DempsterShafer(normalized_reference_np, normalized_sampling_np, is_uncertain)
dsi.hypothesis_order(test_hypotheses)
a = dsi.result()
print(a)


[[0.90817742 0.40032692]
 [0.9825667  0.22219666]
 [1.         0.        ]
 [0.44544484 0.91391433]
 [0.94066825 0.45954704]
 [0.81796729 0.53188908]]
[[0. 1.]]
final mass:  [0.0891785  0.07238459 0.06242241 0.64552518 0.09308772 0.03740159]
     1D CNN      LSTM    BiLSTM  CNN-LSTM       GRU       ANN
0  0.089179  0.072385  0.062422  0.645525  0.093088  0.037402


## Initiate DSI